In [1]:
import sys, os

print("Environment check goes brrr...")
assert(sys.executable.split('/')[-3] == 'conda-env')
assert(os.environ.get('ENVIRONMENT_LOADED') == 'roombot')


Environment check goes brrr...


In [4]:
import llm, sqlite_utils

In [5]:
db = sqlite_utils.Database("./my-embeddings.db")

In [6]:
collection = llm.Collection("entries", db, model_id="3-small")

In [10]:
collection.embed("1", "testing testing 123", store=True, metadata={})